In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup as BS
from datetime import datetime

import pandas as pd
import time
import sys, os
import re
import json, requests


def find_css(css_selector, browser):
    return browser.find_element(By.CSS_SELECTOR, css_selector)
def finds_css(css_selector, browser):
    return browser.find_elements(By.CSS_SELECTOR, css_selector)

def find_xpath(xpath, browser):
    return browser.find_element(By.XPATH, xpath)
def finds_xpath(xpath, browser):
    return browser.find_elements(By.XPATH, xpath)

def find_id(e_id, browser):
    return browser.find_element(By.ID, e_id)

def find_className(cn, browser):
    return browser.find_element(By.CLASS_NAME, cn)
def finds_className(cn , browser):
    return browser.find_elements(By.CLASS_NAME, cn)

def find_linktext(lt, browser):
    return browser.find_element(By.LINK_TEXT, lt)

def find_name(name, browser):
    return browser.find_element(By.NAME, name)
def finds_name(name, browser):
    return browser.find_elements(By.NAME, name)

def find_tagName(tag_name, browser):
    return browser.find_element(By.TAG_NAME, tag_name)

def finds_tagName(tag_name, browser):
    return browser.find_elements(By.TAG_NAME, tag_name)

def resource_path(relative_path):
    """ Get absolute path to resource, works for dev and for PyInstaller """
    base_path = getattr(sys, '_MEIPASS', os.path.dirname(os.path.abspath(__file__)))
    return os.path.join(base_path, relative_path)

In [2]:
def open_browser():
    options = webdriver.ChromeOptions()
    options.add_argument('--no--sandbox')
    options.add_argument('no-sandbox')
    options.add_argument('--disable-dev-shm-suage')
    options.add_argument('--window-size=1080,800')
    options.add_argument('incognito')
    chrome_service = Service('chromedriver')
    chrome_service = Service(executable_path='chromedriver.exe')
    
    browser = webdriver.Chrome(service=chrome_service, options=options)
    
    return browser

In [3]:
def get_url(browser, url):
    browser.get(url)
    time.sleep(1)

In [4]:
df = pd.read_excel('전국_상장회사_data.xlsx', 'seoul_data')

In [5]:
url_list = df['홈페이지'].to_list()
company_name_list = df['회사명'].to_list()

In [6]:
print(len(url_list))
print(len(company_name_list))

1684
1684


In [7]:
url_list_test = url_list[1300:1401]
company_name_list_test = company_name_list[1300:1401]

In [ ]:
data = {}
count = 0

hdr = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'}


for url, company_name in zip(url_list, company_name_list):
    try:
        count2 = 0
        while True:
            response = requests.get(url, headers=hdr)
            count2 += 1
            if count2 == 10:
                break
            if response.status_code == 200:
                soup = BS(response.text, 'html.parser')
                if soup.find_all('html'):
                    page_text = soup.get_text()
                    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
                    email_formats = re.findall(email_pattern, page_text)
                    break

            else:
                print('Unable to access the homepage')
                break

        data[f'{company_name}'] = []
        for email in email_formats:
            data[f'{company_name}'].append({
                email
            })
        count += 1
        print(f'{count} : {company_name} / {email_formats}')
    except Exception as ex:
        pass

1 : 제이엔비 / ['jnb@jnbeng.com']
2 : 동인기연 / ['info@dong-in.com', 'info@dong-in.com']
3 : 에코아이 / ['info@dong-in.com', 'info@dong-in.com']
4 : 스톰테크 / ['bill@stormtec.co.kr', 'sales@stormtec.co.kr']
5 : 캡스톤파트너스 / ['E-MAILmaster@cspartners.co.kr', 'INVESTMENTbiz@cspartners.co.kr']
6 : 프로젠 / ['ir@progen.co.kr']
7 : 에이직랜드 / ['ir@progen.co.kr']
8 : 비아이매트릭스 / []
9 : 쏘닉스 / ['sawsales@sawnics.com']
10 : 세니젠 / []
11 : 유투바이오 / []
12 : 퀄리타스반도체 / ['sales@q-semi.com', 'recruit@q-semi.com', 'ir@q-semi.com']
13 : 미쥬 / []
14 : 워트 / ['wot8600@wot.co.kr']
15 : 에스엘에스바이오 / []
16 : 아이엠티 / ['inquiry.imt@gmail.com', 'sales_imt@imt-c.co.kr', 'inquiry.imt@gmail.com', 'sales_imt@imt-c.co.kr']
17 : 레뷰코퍼레이션 / ['help@revu.net']
18 : 두산로보틱스 / ['help@revu.net']
19 : 한싹 / []
20 : 밀리의서재 / []
21 : 인스웨이브시스템즈 / []
22 : 코어라인소프트 / ['Esales@corelinesoft.com', 'Eobd@corelinesoft.com', 'Esupport@corelinesoft.com', 'Eir@corelinesoft.com', 'Einfo@corelinesoft.eu', 'Fusa@corelinesoft.com']
23 : 율촌 / []
24 : 한화플러스제4호스팩 / []
25 : 크라우드웍

In [165]:
data

{'제이엔비': [{'jnb@jnbeng.com'}],
 '동인기연': [{'info@dong-in.com'}, {'info@dong-in.com'}],
 '에코아이': [{'info@dong-in.com'}, {'info@dong-in.com'}],
 '스톰테크': [{'bill@stormtec.co.kr'}, {'sales@stormtec.co.kr'}],
 '캡스톤파트너스': [{'E-MAILmaster@cspartners.co.kr'},
  {'INVESTMENTbiz@cspartners.co.kr'}],
 '프로젠': [{'ir@progen.co.kr'}],
 '에이직랜드': [{'ir@progen.co.kr'}],
 '비아이매트릭스': [],
 '쏘닉스': [{'sawsales@sawnics.com'}],
 '세니젠': [],
 '유투바이오': [],
 '퀄리타스반도체': [{'sales@q-semi.com'}, {'recruit@q-semi.com'}, {'ir@q-semi.com'}],
 '미쥬': [],
 '워트': [{'wot8600@wot.co.kr'}],
 '에스엘에스바이오': [],
 '아이엠티': [{'inquiry.imt@gmail.com'},
  {'sales_imt@imt-c.co.kr'},
  {'inquiry.imt@gmail.com'},
  {'sales_imt@imt-c.co.kr'}],
 '레뷰코퍼레이션': [{'help@revu.net'}],
 '두산로보틱스': [{'help@revu.net'}],
 '한싹': [],
 '밀리의서재': [],
 '인스웨이브시스템즈': [],
 '코어라인소프트': [{'Esales@corelinesoft.com'},
  {'Eobd@corelinesoft.com'},
  {'Ecsd@corelinesoft.com'},
  {'Eir@corelinesoft.com'},
  {'Einfo@corelinesoft.eu'},
  {'Fusa@corelinesoft.com'}],
 '율촌': []

In [171]:
rows = []
for company, emails in data.items():
    for email_set in emails:
        for email in email_set:
            rows.append({'company_name': company, 'Email': email})

In [167]:
final_df = pd.DataFrame(rows)

In [172]:
final_df = final_df.drop_duplicates(subset='company_name', keep='first')
print(final_df)

    company_name                          Email
0           제이엔비                 jnb@jnbeng.com
1           동인기연               info@dong-in.com
3           에코아이               info@dong-in.com
5           스톰테크            bill@stormtec.co.kr
7        캡스톤파트너스  E-MAILmaster@cspartners.co.kr
..           ...                            ...
748      오리엔트바이오     deokgyeom.kim@orient.co.kr
749         한신공영              gssa@hanshinc.com
755         신라교역            webmaster@sla.co.kr
756        GS글로벌             jgmoon@gsgcorp.com
759       태양금속공업       taeyang@taeyangmetal.com

[398 rows x 2 columns]


In [173]:
final_df.to_excel('test_email2.xlsx', index=False)

In [4]:
browser = open_browser()

In [6]:
start = time.time()
hdr = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'}

url_list = []
count = 1
for i in range(11, 583+1):
    
    url = f'https://www.saramin.co.kr/zf_user/search?search_area=main&search_done=y&search_optional_item=n&loc_mcd=101000&recruitPage={i}&recruitSort=relation&recruitPageCount=100&inner_com_type=&company_cd=0%2C1%2C2%2C3%2C4%2C5%2C6%2C7%2C9%2C10&searchword=&show_applied=&quick_apply=&except_read=&ai_head_hunting=&mainSearch=n'
    response = requests.get(url, headers=hdr)
    if response.status_code == 200:
        soup = BS(response.text, 'html.parser')
        for a_href in soup.find_all(class_='track_event data_layer'):
            url_list.append('https://www.saramin.co.kr'+a_href['href'])
    print(count)
    count += 1
            
url_list = list(set(url_list))
end = time.time()
print(f"{end - start:.5f} sec")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [7]:
len(url_list)

23762

In [8]:
start = time.time()
homepage_urls = []
company_names = []

count = 1

for url in url_list:
    response2 = requests.get(url, headers=hdr)
    
    if response2.status_code == 200:
        soup = BS(response2.text, 'html.parser')

        dt_elements = soup.find_all('dt', string='홈페이지')

        for dt_element in dt_elements:
            dd_element = dt_element.find_next_sibling('dd')
            if dd_element:
                a_tags = dd_element.find_all('a')
                for a_tag in a_tags:
                    homepage_urls.append(a_tag['href'])
                    company_name = soup.find('title').text.split(' ')[0]
                    company_names.append(company_name)
    print(count)
    count += 1
    
end = time.time()
print(f"{end - start:.5f} sec")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699


5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279


ConnectTimeout: HTTPSConnectionPool(host='www.saramin.co.kr', port=443): Max retries exceeded with url: /zf_user/company-info/view?csn=Z2d2ZUpBSDdtQWhyblFaKy91NzVoUT09 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000014B341A4290>, 'Connection to www.saramin.co.kr timed out. (connect timeout=None)'))

In [13]:
ddf = pd.DataFrame(url_list)
ddf.to_excel('seoul_company_homepage_urls.xlsx', index=False)

In [15]:
len(homepage_urls)

3084

In [ ]:
data = {}
count = 0

hdr = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'}


for url,company_name in zip(homepage_urls, company_names):
    try:
        count2 = 0
        while True:
            response = requests.get(url, headers=hdr)
            count2 += 1
            if count2 == 10:
                break
            if response.status_code == 200:
                soup = BS(response.text, 'html.parser')
                if soup.find_all('html'):
                    page_text = soup.get_text()
                    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
                    email_formats = re.findall(email_pattern, page_text)
                    break

            else:
                print('Unable to access the homepage')
                break

        data[f'{company_name}'] = []
        for email in email_formats:
            data[f'{company_name}'].append({
                email
            })
        count += 1
        print(f'{count} : {company_name} / {email_formats}')
    except Exception as ex:
        pass

1 : (주)에이아이디자인건축사사무소 / []
2 : 스타준성형외과피부과 / ['starjun6988@gmail.com']
3 : (주)프리오라에스씨에이 / ['customercare@freora.asia', 'customercare@freora.asia']
4 : (주)보그인터내셔날 / []
5 : (주)글로텍 / ['ebs@mjsoft.com', 'ebs@mjsoft.com']
6 : (주)케이엔에스에듀 / []
7 : (주)아이콘스 / []
8 : 프리미어성형외과 / ['premierps@naver.com', 'premierps@naver.com']
9 : 정동회계법인 / []
10 : 미주강화(주) / ['miju33@chol.com', 'miju33@chol.com']
11 : (주)아크릴 / []
12 : (주)피코팩 / ['cs@picopack.co.kr']
13 : (사)한국한약산업협회 / []
14 : 주식회사 / []
15 : (주)더모션아트텍 / ['motion@themotion.co']
16 : (주)넷트루컨설팅그룹 / ['ncgbiz@netrue.co.kr']
17 : (주)굿미디어 / []
18 : (주)미셀키즈 / []
19 : (주)팬코 / []
20 : 한국테크닉(주) / []
21 : (주)씨앤테크 / []
22 : JK성형외과 / ['webmaster@jkplastic.com', 'webmaster@jkplastic.com', 'parkjk@jk-corp.co.kr', 'webmaster@jkplastic.com']
23 : (주)하이 / ['contact@haii.io']
24 : (주)인코플랜 / []
25 : (주)에이피아이넥스트 / ['sales@api-next.comPhone']
26 : (주)에스앤씨시스템즈 / ['MASTER@HISL.CO.KR']
27 : 퓨처파이프인더스트리스(유) / ['sales-saudiarabia@futurepipe.com', 'sales-saudiarabia@futurepipe.com',

184 : (주)케이에이씨 / []
185 : 서울라헬여성의원 / []
Unable to access the homepage
186 : 한국스트라이커(주) / []
187 : (주)힛더락 / ['-17info@htr.co']
188 : 디아이티(주) / []
189 : 더블유에셋(주) / ['cbs3206@hanmail.net']
190 : (주)더블유메이커스 / []
191 : (주)우일시스템 / []
192 : (주)주용키친플러스 / ['jykp17@daum.net']
193 : (주)산들그린 / []
194 : (주)이공오 / []
195 : (주)티에스티코리아 / ['tstkorea@sungwol.com']
196 : (주)인산가 / ['webmaster@insanga.co.kr', 'imsi2585@insanga.co.kr']
197 : (주)디자인하늘 / []
198 : GAIGAI / []
199 : 해담은풍경건축사사무소 / []
200 : 주식회사 / []
201 : (주)화진화장품 / []
202 : 노보노디스크제약(주) / []
203 : (주)아트숨비 / ['contact@artsoombi.com', 'agency@artsoombi.com']
204 : (주)유일종합조경 / []
205 : (주)제이디큐브릭 / []
206 : (주)에코먼트 / ['tlsckdduq@ecoment.co']
207 : 주식회사비오드 / []
208 : (주)스페이드컴퍼니 / ['contact@spadecompany.kr', 'contact@spadecompany.kr']
Unable to access the homepage
209 : (주)리공교역 / ['contact@spadecompany.kr', 'contact@spadecompany.kr']
210 : 블리비의원 / []
211 : 크리에이티브파트너스 / ['contact@creativepartners.co.kr']
212 : (주)신조엠앤디 / []
213 : (주)워드캣 / []
214 : (주)어셔

407 : (유)베스타스코리아윈드테크놀러지 / []
408 : 케이엔티인베스트먼트(주) / ['knt@kntp.co.kr', 'knt@kntp.co.kr']
409 : (주)디럭스뷰티 / ['ynm@ynmcosmetic.com']
410 : (주)의식주컴퍼니 / ['business@lifegoeson.kr', 'jhlim@lifegoeson.kr']
411 : (주)케이비에스시큐리티 / ['kbsscr@kbssecurity.co.kr']
412 : Study / []
413 : 더봄플러스주식회사 / ['thebomplus@naver.com']
414 : (주)영진어패럴 / []
415 : (주)담꽃 / ['damkkot@naver.com']
416 : (주)그린랩스 / []
417 : (주)제이앤지시스템 / ['sales@jngsystem.co.kr', 'jng@jngsystem.co.kr']
418 : (주)라피끄 / ['sales@rafiqcosmetics.com']
419 : (주)무브 / []
420 : (주)브랜드501 / ['agent@brand501.com']
421 : (주)싸이토젠 / ['CUSTOMER@CYTOGENLAB.COM']
422 : (주)크로스앵글 / []
423 : (주)뱅코 / []
424 : (주)개성건설 / ['kscourse@naver.com']
425 : (주)송아리아이티 / []
426 : (주)엘에스케이글로벌파마서비스 / ['Lee.youngbum@lskglobal.com', 'Lee.youngbum@lskglobal.com', 'Lee.youngbum@lskglobal.com', 'Lee.youngbum@lskglobal.com']
427 : (사)한국중소기업발전협회 / ['manager@ceo-korea.org']
428 : (주)웹트리 / ['filemonghelp@gmail.com', 'filemonghelp@gmail.com', 'filemonghelp@gmail.com']
Unable to access th

562 : (주)탑로지스틱스코리아 / ['topoceaninfo@topocean.com']
563 : (주)포크홀딩스 / []
564 : (주)에이랜드 / ['online@alandcompany.com']
565 : 브이앤드온코리아(주) / []
566 : (주)고반홀딩스 / []
567 : (주)아카이브코퍼레이션 / []
568 : (주)얼티밋커리어그룹 / []
569 : (주)우전 / []
570 : (주)성산테크 / []
571 : (주)핌아시아 / []
Unable to access the homepage
572 : 엠엠에스(주) / []
573 : (주)세아에프에스 / []
574 : (주)클라모스 / ['info@clamos.iobottom']
575 : 한국머스크(주) / []
576 : (주)그로스체인 / ['reserved.contact@growthchain.com']
577 : 승일운수(주) / ['reserved.contact@growthchain.com']
578 : (주)비자레이매지네이션 / []
579 : (주)올댓페이 / []
580 : (주)짐싸 / []
581 : (주)야다 / ['info@yadahcosmetics.com', 'overseas@yadahcosmetics.com']
582 : (주)리모프렌즈 / ['moon@mo-on.co.kr']
583 : 메가스터디(주) / []
584 : (주)유원기획 / []
585 : (주)피오피플랜 / ['pop4@popplan.co.kr']
586 : 하이텍앤솔(주) / ['qna@hitecnsol.com']
587 : (주)리얼클릭 / []
588 : 한국기술신용평가(주) / ['help@k-tcb.com']
589 : (주)이건창호 / []
590 : 헥사곤지엠아이(주) / ['apply@hexagongmi.com']
591 : (주)제이와이드컴퍼니 / []
592 : (주)제닉스 / []
593 : (주)아하빌딩부동산중개법인 / ['yt-support-solutions-kr@go

792 : (주)위드컬처 / []
793 : 한국진로적성센터 / []
794 : K&S / []
795 : 지방자치연구소(주) / []
Unable to access the homepage
796 : 리팅성형외과의원(부산) / []
797 : (주)인큐테크 / []
798 : (주)랩투마켓 / ['namuook@lab2m.com']
799 : (주)액트파이브 / ['info@act5.co.kr']
800 : (주)비티인터내쇼날손해보험중개 / ['2880Emailinfo@btisko.com']
801 : (주)골든어스 / ['tech@goldenus.co.kr']
802 : 데브시스터즈(주) / []
803 : 스카이서울안과의원 / []
804 : 9치한의원 / ['000@0000.com']
805 : 케이피(KPEE)필라테스 / ['nightsp@naver.com']
806 : (주)인핸스파트너스 / ['mspark@enhancepartners.co.kr']
807 : 덴츠플라이시로나코리아(유) / []
808 : (주)대동메디칼컨설팅 / ['info@dmci.co.kr', 'info@dmci.co.kr']
809 : 기프트레터(주) / ['mkt@gift-n.co.kr', 'mkt@gift-n.co.kr']
810 : (주)메디플렉서스 / []
811 : (주)모드하우스 / ['cs@mod-haus.comPhone', 'business@mod-haus.com']
812 : (주)제이엠더블유 / ['jmwkorea@jmwkorea.com']
813 : (주)엘에스컴퍼니 / ['cs@lscompany.co.kr']
814 : (주)다난기술 / []
815 : (주)디오에프연구소 / []
816 : 그릿모터테인먼트(주) / ['brian.lee@gritmotortainment.com']
817 : 세기밀리너(주) / ['209-81-26343info@thesegi.com']
818 : (주)이트라이브 / []
819 : (주)엑토 / []
820 : (주)에이비씨

1032 : (주)그린에스씨에이 / []
1033 : (주)테크원씨엔에스 / ['techonecns@naver.com']
1034 : 르노코리아자동차송파대로대리점 / ['techonecns@naver.com']
1035 : 세종파츠플러스(유) / ['sejongparts@sejongparts.com']
1036 : 제이시스 / []
1037 : 범양공조산업(주) / []
1038 : (주)삼현 / []
1039 : (주)지알케이파트너스 / ['items.admin@goldenrakun.comPath', '45admin@goldenrakun.comFax']
1040 : 대진반도체(주) / ['items.admin@goldenrakun.comPath', '45admin@goldenrakun.comFax']
1041 : (주)크레온유니티 / []
1042 : 한국삼공(주) / []
1043 : 제이큐브데이터(주) / []
1044 : 선일티앤티(주) / ['cs@sunilline.com', 'pumper21@kobiznet.co']
1045 : 비손메디칼(주) / ['sales@bisonmedical.com']
1046 : (주)비즈테크아이 / []
1047 : (주)서버몬 / []
1048 : (주)케이씨인더스트리얼 / []
1049 : 코리아휴먼리스오스시스템(주) / []
1050 : (주)싸이몬 / []
1051 : 잡스테이션코리아 / ['job@jobstationkorea.com']
1052 : (주)웨슬리퀘스트 / []
1053 : 디에스글로벌(주) / []
1054 : 에듀플러스(주) / []
1055 : (주)잇츠비솔루션 / []
1056 : (주)아가방앤컴퍼니 / []
1057 : (주)정호티엘씨 / []
1058 : (재)한국의학연구소 / ['webmaster@kmi.or.kr']
1059 : 동림건설 / ['cho9798@tsis.co.kr', 'sunkyoungpark@tsis.co.kr']
1060 : 한국맥널티(주) / []
1061 : 세민

1255 : (주)파인드디앤서 / ['findther@naver.com']
1256 : 엠에스인텍(주) / []
1257 : 연세라미슈의원 / []
1258 : (주)아이로지스코리아 / []
1259 : (주)세라컴 / []
1260 : (주)셈텍스 / ['info@samtexglobal.com']
1261 : (주)제이알인터내셔널에이지 / ['jr@jrinter.kr']
1262 : (주)씨엠월드 / []
1263 : 야나 / ['yarn-a@naver.com']
1264 : 롯데정보통신(주) / []
1265 : 에이지엘(주) / []
1266 : (주)일신이앤씨 / []
1267 : 숙명여자대학교산학협력단 / []
1268 : 디자인이음새(주) / []
1269 : (주)더온에프앤비 / []
1270 : (주)큐리오시스 / ['sales@curiosis.com']
1271 : (주)동서 / []
1272 : (주)서울스칼라스에듀 / ['info@ssiand.org', 'info@ssiand.org', 'info@ssiand.org']
1273 : (주)케이티씨에스 / ['info@ssiand.org', 'info@ssiand.org', 'info@ssiand.org']
1274 : (주)파고다아카데미 / []
1275 : (주)티제이팜 / []
1276 : (주)다온 / []
1277 : (주)우리씨드 / []
1278 : 라메종쥬코리아(주) / ['maisonjus@jusparfums.co.kr']
1279 : 이가자미용학원 / ['leekaja3333433@gmail.com', 'leekaja3333433@gmail.com']
1280 : (주)신티에스 / []
1281 : (주)메디잡리더스 / []
1282 : (주)삼십육점오그라픽스 / []
1283 : 셀로이드(주) / ['emailinfo@celloid.co.krContact', 'emailinfo@celloid.co.krCopyright']
1284 : (주)씨메스 / ['Sales@cmesr

Unable to access the homepage
1479 : 써밋투어(주) / ['contact@coconutsilo.comCustomer', 'contact@coconutsilo.com']
1480 : (주)우루루 / []
1481 : 미래글로넷(주) / []
1482 : (주)금성전력기술 / []
1483 : 디딤365(주) / ['sales@didim365.com', 'support@didim365.com']
1484 : 쥴릭파마코리아(주) / []
1485 : 서초여성가족플라자서초센터 / []
1486 : 농심엔지니어링(주) / []
1487 : (주)이터샘디자인 / []
1488 : (주)신펙스커넥션로지스틱 / []
1489 : (주)루닛 / ['contact@lunit.io', 'ir@lunit.io', 'media@lunit.io', 'partner@lunit.io', 'apply@lunit.io']
1490 : (주)투모로네트웍 / ['contact@lunit.io', 'ir@lunit.io', 'media@lunit.io', 'partner@lunit.io', 'apply@lunit.io']
1491 : (주)디에프코퍼레이션 / ['kjy@firstcanvas.co.kr', 'kjy@firstcanvas.co']
1492 : 이화전기공업(주) / ['sales@eti21.com']
1493 : (주)레몬클라우드 / ['lemon@lemoncloud.ioCopyright', 'steve@lemoncloud.io']
1494 : (주)케이비저축은행 / []
1495 : (주)국순당 / []
1496 : (주)아이오티스페이스 / ['info@iotspace.co.kr']
1497 : 시그네틱스(주) / []
1498 : 주식회사 / ['help@inroes.co']
1499 : (주)파마폰티스 / ['help@inroes.co']
1500 : 아이정(주) / ['service@ijung.co.kr', 'service@ijung.co.kr']
1

1690 : (주)평일 / []
1691 : 신한카드(주) / []
1692 : 스프링거네이처코리아(유) / []
1693 : (주)하이웨이원 / []
1694 : 실로암장애인근로사업장 / []
1695 : 케이티엘글로벌로지스틱스(주) / []
1696 : (주)호프인터내셔널 / ['hopeintl@hope-intl.com']
1697 : 씨스존(주) / []
1698 : 희망노인복지센타 / []
1699 : (주)클라메드 / ['key@clamed.co.kr', 'key@clamed.co.kr']
1700 : (주)그린패키지솔루션 / ['reply@gps-pack.com', 'reply@gps-pack.com']
1701 : (주)마이티일렉 / []
1702 : (주)한결피아이에프 / []
1703 : (주)지에스디자인 / ['docdh@hanmail.net', 'docdh@hanmail.net']
1704 : 주액터스연극학원 / []
1705 : (주)수젠텍 / ['Info@sugentech.com', 'E-mailir@sugentech.com']
1706 : 에린디자인 / []
1707 : (주)씨에스콤프텍 / []
1708 : 더 / ['info@therye.kr']
1709 : 건설기업(주) / []
1710 : (주)신일스틸 / []
1711 : 아이빔테크놀로지(주) / ['information@ivimtech.com', 'information@ivimtech.com', 'information@ivimtech.com', 'information@ivimtech.com']
1712 : (주)엔코스 / ['ancorsmk@gmail.com']
1713 : 대현세무법인 / []
1714 : (주)드림업컨설팅 / []
1715 : 코레일테크(주) / ['1234-567-8902office@example.comcontact']
1716 : (주)유투바이오 / []
1717 : (주)디뉴미디어 / []
1718 : (주)생각하는교육 / []
1719 : 람다리서

In [82]:
rows = []
for company, emails in data.items():
    for email_set in emails:
        for email in email_set:
            rows.append({'company_name': company, 'Email': email})
            
final_df = pd.DataFrame(rows)
final_df = final_df.drop_duplicates(subset='company_name', keep='first')
print(final_df)

    company_name                     Email
0       (주)아이엔티랩        chief@intlab.co.kr
1         (주)씨메스     Sales@cmesrobotics.ai
4       (주)한건이엔씨  creativehg@naver.comAddr
5        (주)이퀴닉스     champhelp@hackers.com
6       (주)챔프스터디        soonsoof@naver.com
..           ...                       ...
310  (주)맨파워그룹코리아           sd90@sd90.co.kr
312     보울박스한남본점       webmaster@t-ime.com
313      신성통상(주)     firstactors@naver.com
314        네오리소스     blue@bluemeditech.com
315     (주)정직한도움     blue@bluemeditech.com

[191 rows x 2 columns]


In [83]:
final_df.to_excel('test_email3.xlsx', index=False)

--------------

-------------

In [34]:
email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
email_formats = re.findall(email_pattern, page_text)

for email in email_formats:
    print(email)

In [8]:
main = browser.window_handles

for i in main:
    if i != main[0]:
        browser.switch_to.window(i)
        browser.close()

jnb@jnbeng.com


In [15]:
soup = BS(browser.page_source, 'html.parser')

footer_text = soup.find('footer').get_text() if soup.find('footer') else None
print(footer_text)



(주)제이엔비 17708 경기도 평택시 진위면 진위2산단로 31-21
TEL. 031)8015-2001    FAX. 031)8015-2005    MAIL. jnb@jnbeng.com

COPYRIGHT 2021⒞ JNB, ALL RIGHTS RESERVED.




In [10]:
browser = open_browser()

for url in url_list:
    get_url(browser, url)
    soup = BS(browser.page_source, 'html.parser')

    footer_text = soup.find('footer').get_text() if soup.find('footer') else None
    print(footer_text)
    print('--' * 20)
    time.sleep(1)



(주)제이엔비 17708 경기도 평택시 진위면 진위2산단로 31-21
TEL. 031)8015-2001    FAX. 031)8015-2005    MAIL. jnb@jnbeng.com

COPYRIGHT 2021⒞ JNB, ALL RIGHTS RESERVED.


----------------------------------------


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=121.0.6167.161)
Stacktrace:
	GetHandleVerifier [0x00007FF6B64A5E42+3538674]
	(No symbol) [0x00007FF6B60C4C02]
	(No symbol) [0x00007FF6B5F75AEB]
	(No symbol) [0x00007FF6B5F5288C]
	(No symbol) [0x00007FF6B5FE5DD7]
	(No symbol) [0x00007FF6B5FFB40F]
	(No symbol) [0x00007FF6B5FDEE53]
	(No symbol) [0x00007FF6B5FAF514]
	(No symbol) [0x00007FF6B5FB0631]
	GetHandleVerifier [0x00007FF6B64D6CAD+3738973]
	GetHandleVerifier [0x00007FF6B652C506+4089270]
	GetHandleVerifier [0x00007FF6B6524823+4057299]
	GetHandleVerifier [0x00007FF6B61F5C49+720121]
	(No symbol) [0x00007FF6B60D126F]
	(No symbol) [0x00007FF6B60CC304]
	(No symbol) [0x00007FF6B60CC432]
	(No symbol) [0x00007FF6B60BBD04]
	BaseThreadInitThunk [0x00007FFECD217344+20]
	RtlUserThreadStart [0x00007FFECE4426B1+33]


footer을 가져오는 것은 None가 너무 많이 발생해 Pass
다른 방안을 고안

In [92]:
get_url(browser ,url_list[3])

In [95]:
company_name_list[3]

'스톰테크'

In [14]:
soup = BS(browser.page_source, 'html.parser')

# "@"이 포함된 텍스트만 출력
# 정규식 패턴
email_pattern = re.compile(r'^[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$')
email_pattern2 = re.compile(r'^.*[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$')

elements = soup.get_text()
elements = elements.split('\n')

data = {}


for i in elements:
    matches = re.findall(email_pattern2, i)
    if len(matches) >= 1:
        print(matches)

['TEL. 031)8015-2001\xa0\xa0\xa0\xa0FAX. 031)8015-2005\xa0\xa0\xa0\xa0MAIL. jnb@jnbeng.com']
['TEL. 031)8015-2001\xa0\xa0\xa0\xa0FAX. 031)8015-2005\xa0\xa0\xa0\xa0MAIL. jnb@jnbeng.com']


In [14]:
def add_email_info(company, email):
    try:
        with open('email_info.json', 'r', encoding='utf-8') as json_file:
            email_info = json.load(json_file)
    except FileNotFoundError:
        # 파일이 없는 경우 초기화
        email_info = {}

    # 기존 정보에 새로운 정보 추가
    email_info[company] = email

    # JSON 형식으로 저장
    with open('email_info.json', 'w', encoding='utf-8') as json_file:
        json.dump(email_info, json_file, ensure_ascii=False, indent=2)

    print(f'{company}의 이메일 정보가 추가되었습니다.')

In [ ]:
browser = open_browser()

for url, company_name in zip(url_list, company_name_list):
    try:
        get_url(browser, url)

        soup = BS(browser.page_source, 'html.parser')

        # "@"이 포함된 텍스트만 출력
        # 정규식 패턴
        email_pattern = re.compile(r'^[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$')
        email_pattern2 = re.compile(r'^.*[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$')

        elements = soup.get_text()
        elements = elements.split('\n')

        data = {}


        for i in elements:
            matches = re.findall(email_pattern2, i)
            if len(matches) >= 1:
                email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
                emails = re.findall(email_pattern, matches[0])
                add_email_info(company, emails)
#                 print(f"{company_name}: {emails}")
    except:
        pass

In [12]:
matches = ['Robots for Every Workplace쉽고 경제적인 협동로봇으로 안전하고 편리한 자동화를 선도합니다.Neuromeka.협동로봇의 다양한 활용을 통해 제조공정의 생산성 향상, F&B 서비스의 안정성 확대 그리고 문화예술 산업의 새로운 가능성을 뉴로메카가 함께합니다.Robot.협동로봇과 델타로봇 등 다양한 로봇 라인업을 통해 생산성 향상과 원가 절감 효과를 기대할 수 있으며 동시에 산업재해 고민도 해결할 수 있습니다.Video.동영상 보기동영상 보기01:20동영상 보기동영상 보기02:17동영상 보기동영상 보기01:44동영상 보기동영상 보기01:19동영상 보기동영상 보기03:48동영상 보기동영상 보기02:08동영상 보기동영상 보기04:39Post.News.7일 전뉴로메카, SK네트웍스서비스와 로봇 공급 총판 계약..."기간은 2년, 이후엔 갱신"12월 13일지능로봇 리빙랩 기반 인재육성 나선다…수원시 협약11월 10일교촌 치킨을 뉴로메카 로봇이 만든다12345Robot as a Tool |\xa0Robot as a Service |\xa0Robots for Every Workplace\u200b(주)뉴로메카\xa0 \xa0 \xa0서울특별시 성동구 아차산로 78, 에코넷센터 (04782) \xa0 \xa0 TEL : 1661-0773\xa0 \xa0 \xa0영업 : sales@neuromeka.com\xa0 \xa0 \xa0마케팅/홍보 : pr@neuromeka.com']

In [13]:
email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
emails = re.findall(email_pattern, matches[0])
print(emails)

['sales@neuromeka.com', 'pr@neuromeka.com']


In [10]:
email_addresses = []

for a_tag in soup.find_all('a', href=True):
    if 'mailto:' in a_tag['href']:
        email_addresses.append(a_tag['href'].split(':')[1])

for span_tag in soup.find_all('span'):
    email_addresses.append(span_tag.get_text())

# 결과 출력
# print(email_addresses)
for i in email_addresses:
    matches = re.findall(email_pattern, i)
    print(matches)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [18]:
from bs4 import BeautifulSoup

# 예시로 사용할 HTML
html = """
<html>
  <body>
    <p>This is the first paragraph.</p>
    <p>This is the second paragraph.</p>
    <p>This is the third paragraph.</p>
  </body>
</html>
"""

# BeautifulSoup 객체 생성
soup = BeautifulSoup(html, 'html.parser')

# 모든 텍스트 가져오기
all_text = soup.get_text()

all_text.split('\n')

['',
 '',
 '',
 'This is the first paragraph.',
 'This is the second paragraph.',
 'This is the third paragraph.',
 '',
 '',
 '']